In [1]:
import sys
 
sys.path.append('..')
from piece import piece

piece.start(should_send_to_score=False)

In [141]:
from soundmining_tools.generative import *
from soundmining_tools.sieve import *
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.sequencer import Sequencer, SequenceNote
from enum import Enum
from typing import Callable
import math
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipycanvas import Canvas

sound_path = "/Users/danielstahl/Documents/Music/Pieces/Ambient Music/Ambient Music 8/sounds/Ambient Music 8 Sounds"

# table hit cardoid 24, 33, 45, 94, 169, 280, 494, 773, 811, 916
# table hit omni 28, 56, 77, 97, 143, 279, 307, 515, 705, 962, 1171, 1500, 1692

piece.reset()
piece.synth_player \
    .add_sound("table_hit_omni", f"{sound_path}/Table Hit 1-Omni mix.flac", 0.169, 0.481) \
    .add_sound("table_hit_cardioid", f"{sound_path}/Table Hit 1-Cardioid mix.flac", 0.169, 0.481) \
    .start()

NR_OF_PARTS = 8
#EFFECT_LENGTH = 120
#EFFECT_LENGTH = 30
EFFECT_LENGTH = NR_OF_PARTS * 60 * 0.75
print(f"Effect length {EFFECT_LENGTH}")

def make_effects(long_effect_output: int, long_clean_output: int, short_effect_output: int, short_clean_output: int):
    long_effect_sound = piece.synth_player.note(NodeId.EFFECT) \
        .stereo_input() \
        
    long_reverb_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
        .input_from_note(long_effect_sound) \
        .stereo_volume(piece.control_instruments.static_control(0.2)) \
        .stereo_g_verb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), roomsize=50, revtime=8, damping=0.7, inputbw=0.5, earlyreflevel=0.5, taillevel=0.7) \
        .stereo_high_pass_filter(piece.control_instruments.static_control(100)) \
        .stereo_low_pass_filter(piece.control_instruments.static_control(1000))
        
    long_reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=long_effect_output)

    long_clean_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
        .input_from_note(long_effect_sound) \
        .stereo_volume(piece.control_instruments.static_control(0.8))
    long_clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=long_clean_output)

    short_effect_sound = piece.synth_player.note(NodeId.EFFECT) \
        .stereo_input() \
        
    short_reverb_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
        .input_from_note(short_effect_sound) \
        .stereo_volume(piece.control_instruments.static_control(0.3)) \
        .stereo_free_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), mix=1.0, room=0.8, damp=0.5)
        #.stereo_g_verb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), roomsize=50, revtime=8, damping=0.7, inputbw=0.5, earlyreflevel=0.5, taillevel=0.7) \
        #.stereo_high_pass_filter(piece.control_instruments.static_control(100)) \
        #.stereo_low_pass_filter(piece.control_instruments.static_control(1000))
        
    short_reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=short_effect_output)

    short_clean_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
        .input_from_note(short_effect_sound) \
        .stereo_volume(piece.control_instruments.static_control(0.7))
    short_clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=short_clean_output)

    return long_effect_sound, short_effect_sound

long_sub_effect, short_sub_effect = make_effects(0, 0, 0, 0)
long_low_effect, short_low_effect = make_effects(0, 0, 0, 0)
long_middle_effect, short_middle_effect = make_effects(0, 0, 0, 0)
long_high_effect, short_high_effect = make_effects(0, 0, 0, 0)

OMNI_SOUND = "table_hit_omni"
CARDIOID_SOUND = "table_hit_cardioid"

SUB_NOISE_TRACK = "Sub noise"
SUB_SOUND_TRACK = "Sub sound"
LOW_NOISE_TRACK = "Low noise"
LOW_SOUND_TRACK = "Low sound"
MIDDLE_NOISE_TRACK = "Middle noise"
MIDDLE_SOUND_TRACK = "Middle sound"
HIGH_NOISE_TRACK = "High noise"
HIGH_SOUND_TRACK = "High sound"
OMNI_COLOR = "red"
CARDIOD_COLOR = "blue"
OMNI_DURATION = piece.synth_player.get_sound(OMNI_SOUND).duration(1.0)
CARDIOID_DURATION = piece.synth_player.get_sound(CARDIOID_SOUND).duration(1.0)

class NoteState:
    def __init__(self, freq_transion_matrix: dict, freqs: list[float]) -> None:
        self._freq_chain = MarkovChain(freq_transion_matrix, 0)
        self._freqs = freqs

    def next_freq(self) -> float:
        return self._freqs[self._freq_chain.next()]

sub_omni_transition_matrix = {
    0: {1: 0.66, 0: 0.33},
    1: {0: 0.66, 1: 0.33},
}
sub_omni_freqs = [28, 56]
sub_omni_state = NoteState(sub_omni_transition_matrix, sub_omni_freqs)

def table_hit_sub_omni(start_time: float) -> SequenceNote:
    rq = random_range(0.05, 0.09)
    band_pass = sub_omni_state.next_freq()
    start = start_time + random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono("table_hit_omni", 1, piece.synth_player.control_instruments.static_control(random_range(15, 17))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(56)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(2.5)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.3, 0.3))) \
        .send_to_synth_note(short_sub_effect, start)
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start, track=SUB_SOUND_TRACK, duration=OMNI_DURATION, freq=band_pass, color=OMNI_COLOR)

sub_omni_noise_state = NoteState(sub_omni_transition_matrix, sub_omni_freqs)

def sub_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = sub_omni_noise_state.next_freq()    
    amp_control = piece.control_instruments.three_block_control((0, random_range(3, 5), random_range(3, 5), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(0.4, [(-0.3, 0.3)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.9, 1.5)), 56, 65)
    duration = base_duration * random_range(0.85, 1.15)
    start = start_time + random_range(0.1, 0.5)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_sub_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=SUB_NOISE_TRACK, duration=duration, freq=band_pass, color=OMNI_COLOR)


sub_cardioid_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}
sub_cardioid_freqs = [24, 33, 45]
sub_cardioid_state = NoteState(sub_cardioid_transition_matrix, sub_cardioid_freqs)

# 0.169, 0.481
def table_hit_sub_cardioid(start_time: float) -> SequenceNote:
    rq = random_range(0.1, 0.2)
    band_pass = sub_cardioid_state.next_freq()    
    start = start_time + random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono("table_hit_cardioid", 1, piece.synth_player.control_instruments.static_control(random_range(2, 4)), end_override=0.350) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(45)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(5)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.3, 0.3))) \
        .send_to_synth_note(short_sub_effect, start)
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start, track=SUB_SOUND_TRACK, duration=CARDIOID_DURATION, freq=band_pass, color=CARDIOD_COLOR)

sub_cardioid_noise_state = NoteState(sub_cardioid_transition_matrix, sub_cardioid_freqs)

def sub_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = sub_cardioid_noise_state.next_freq()    
    amp_control = piece.control_instruments.three_block_control((0, random_range(3, 5), random_range(3, 5), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(0.4, [(-0.3, 0.3)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.9, 1.5)), 45, 55)
    start = start_time + random_range(0.1, 0.5)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_sub_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=SUB_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)


low_omni_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}
low_omni_freqs = [77, 97, 143, 279]
low_omni_state = NoteState(low_omni_transition_matrix, low_omni_freqs)

def table_hit_low_omni(start_time: float)  -> SequenceNote: 
    rq = random_range(0.5, 0.9)
    band_pass = low_omni_state.next_freq()
    start = start_time + random_range(0.01, 0.02)
    piece.synth_player.note() \
        .sound_mono("table_hit_omni", 1, piece.synth_player.control_instruments.static_control(random_range(1, 3))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(77)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(279)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.5, 0.5))) \
        .send_to_synth_note(short_low_effect, start)
        #.play(start_time + random_range(0.01, 0.02))
    return SequenceNote(start=start, track=LOW_SOUND_TRACK, duration=OMNI_DURATION, freq=band_pass, color=OMNI_COLOR)


low_omni_noise_state = NoteState(low_omni_transition_matrix, low_omni_freqs)

def low_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = low_omni_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.8, 1.3), random_range(0.8, 1.3), 0), (0.1, 0.8, 0.1), (4, 0, -4))
    pan_start, pan_end = pan_line(0.6, [(-0.5, -0.25), (0.25, 0.5)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.1, 0.3)), 279, 300)
    duration = base_duration * random_range(0.85, 1.15)
    start = start_time + random_range(0.1, 0.5)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(77)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_low_effect, start, duration)
        #.play(start, duration)
    return SequenceNote(start=start, track=LOW_NOISE_TRACK, duration=duration, freq=band_pass, color=OMNI_COLOR)

low_cardioid_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}
low_cardioid_freqs = [94, 169, 280]
low_cardioid_state = NoteState(low_cardioid_transition_matrix, low_cardioid_freqs)

def table_hit_low_cardioid(start_time: float)  -> SequenceNote: 
    rq = random_range(0.5, 0.9)
    band_pass = low_cardioid_state.next_freq()
    start = start_time + random_range(0.001, 0.005)
    piece.synth_player.note() \
        .sound_mono("table_hit_cardioid", 1, piece.synth_player.control_instruments.static_control(random_range(1, 2))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(94)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(280)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.5, 0.5))) \
        .send_to_synth_note(short_low_effect, start)
        #.play(start_time + random_range(0.001, 0.005))
    return SequenceNote(start=start, track=LOW_SOUND_TRACK, duration=CARDIOID_DURATION, freq=band_pass, color=CARDIOD_COLOR)                  

low_cardioid_noise_state = NoteState(low_cardioid_transition_matrix, low_cardioid_freqs)

def low_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = low_cardioid_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.8, 1.3), random_range(0.8, 1.3), 0), (0.1, 0.8, 0.1), (4, 0, -4))
    pan_start, pan_end = pan_line(0.6, [(-0.5, -0.25), (0.25, 0.5)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.1, 0.3)), 280, 320)
    duration = base_duration * random_range(0.85, 1.15)
    start = start_time + random_range(0.1, 0.5)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(94)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_low_effect, start, duration)
        #.play(start, duration)
    return SequenceNote(start=start, track=LOW_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)
    
middle_omni_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}
middle_omni_freqs = [307, 515, 705]
middle_omni_state = NoteState(middle_omni_transition_matrix, middle_omni_freqs)

def table_hit_middle_omni(start_time: float)  -> SequenceNote: 
    rq = random_range(0.5, 0.9)
    band_pass = middle_omni_state.next_freq()
    start = start_time + random_range(0.001, 0.005)
    piece.synth_player.note() \
        .sound_mono("table_hit_omni", 1, piece.synth_player.control_instruments.static_control(random_range(0.3, 0.5))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(307)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(705)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(3)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.75, -0.5), (0.5, 0.75)]))) \
        .send_to_synth_note(short_middle_effect, start)
        #.play(start_time + random_range(0.001, 0.005))
    return SequenceNote(start=start, track=MIDDLE_SOUND_TRACK, duration=OMNI_DURATION, freq=band_pass, color=OMNI_COLOR)

middle_omni_noise_state = NoteState(middle_omni_transition_matrix, middle_omni_freqs)

def middle_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = middle_omni_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.5, 2), random_range(0.5, 2), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(0.8, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.5, 0.75)), 705, 740)
    start = start_time + random_range(0.1, 0.5)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(307)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_middle_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=MIDDLE_NOISE_TRACK, duration=duration, freq=band_pass, color=OMNI_COLOR)
    
middle_cardiod_transition_matrix = {
    0: {1: 0.7, 1: 0.3},
    1: {0: 0.7, 1: 0.3}
}
middle_cardioid_freqs = [494, 773]
middle_cardioid_state = NoteState(middle_cardiod_transition_matrix, middle_cardioid_freqs)

def table_hit_middle_cardioid(start_time: float)  -> SequenceNote: 
    rq = random_range(0.5, 0.9)
    band_pass = middle_cardioid_state.next_freq()
    start = start_time + random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono("table_hit_cardioid", 1, piece.synth_player.control_instruments.static_control(random_range(0.3, 0.5))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(494)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(773)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(3)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.75, -0.5), (0.5, 0.75)]))) \
        .send_to_synth_note(short_middle_effect, start)
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start, track=MIDDLE_SOUND_TRACK, duration=CARDIOID_DURATION, freq=band_pass, color=CARDIOD_COLOR)

middle_cardioid_noise_state = NoteState(middle_cardiod_transition_matrix, middle_cardioid_freqs)

def middle_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = middle_cardioid_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.5, 2), random_range(0.5, 2), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(0.8, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.5, 0.75)), 773, 780)
    start = start_time + random_range(0.1, 0.5)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(494)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_middle_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=MIDDLE_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)
    
high_omni_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}
high_omni_freqs = [962, 1171, 1500, 1692]
high_omni_state = NoteState(high_omni_transition_matrix, high_omni_freqs)

def table_hit_high_omni(start_time: float)  -> SequenceNote:      
    rq = random_range(0.5, 0.9)
    band_pass = high_omni_state.next_freq()
    start = start_time + random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono("table_hit_omni", 1, piece.synth_player.control_instruments.static_control(random_range(0.3, 0.5))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(962)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(1692)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(2)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.99, -0.75), (0.75, 0.99)]))) \
        .send_to_synth_note(short_high_effect, start)
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start, track=HIGH_SOUND_TRACK, duration=OMNI_DURATION, freq=band_pass, color=OMNI_COLOR)

high_omni_noise_state = NoteState(high_omni_transition_matrix, high_omni_freqs)

def high_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = high_omni_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.5, 2), random_range(0.5, 2), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(1.1, [(-0.99, -0.75), (0.75, 0.99)])
    rq = random_range(1, 3)    
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.04, 0.08)), 1692, 1720)
    start = start_time + random_range(0.1, 0.5)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(962)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_high_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=HIGH_NOISE_TRACK, duration=duration, freq=band_pass, color=OMNI_COLOR)
    
high_cardiod_transition_matrix = {
    0: {1: 0.7, 1: 0.3},
    1: {0: 0.7, 1: 0.3}
}
high_cardioid_freqs = [811, 916]
high_cardioid_state = NoteState(high_cardiod_transition_matrix, high_cardioid_freqs)

def table_hit_high_cardioid(start_time: float)  -> SequenceNote: 
    rq = random_range(0.5, 0.9)
    band_pass = high_omni_state.next_freq()
    start = start_time + random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono("table_hit_cardioid", 1, piece.synth_player.control_instruments.static_control(random_range(0.3, 0.5))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(811)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(916)) \
        .mono_volume(piece.synth_player.control_instruments.static_control(4)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.99, -0.75), (0.75, 0.99)]))) \
        .send_to_synth_note(short_high_effect, start)
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start, track=HIGH_SOUND_TRACK, duration=CARDIOID_DURATION, freq=band_pass, color=CARDIOD_COLOR)
        
high_cardioid_noise_state = NoteState(high_cardiod_transition_matrix, high_cardioid_freqs)

def high_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = high_cardioid_noise_state.next_freq()
    amp_control = piece.control_instruments.three_block_control((0, random_range(0.5, 2), random_range(0.5, 2), 0), (0.3, 0.4, 0.3), (4, 0, -4))
    pan_start, pan_end = pan_line(1.1, [(-0.99, -0.75), (0.75, 0.99)])
    rq = random_range(1, 3)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.04, 0.08)), 916, 1100)
    start = start_time + random_range(0.1, 0.5)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(811)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .send_to_synth_note(long_high_effect, start, duration)
        #.play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start, track=HIGH_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)
    
class SampleNoteHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)        

    def handle_note(self, patch_arguments: PatchArguments) -> None:            
        match patch_arguments.octave:
            case 2:
                match patch_arguments.note:  
                    case 0:
                        table_hit_sub_omni(patch_arguments.start)  
                    case 1:
                        table_hit_sub_cardioid(patch_arguments.start)
                    case 2:
                        table_hit_low_omni(patch_arguments.start)                        
                    case 3:
                        table_hit_low_cardioid(patch_arguments.start)                                                    
                    case 4:      
                        table_hit_middle_omni(patch_arguments.start)                                 
                    case 5:                        
                        table_hit_middle_cardioid(patch_arguments.start)                                    
                    case 6:    
                        table_hit_high_omni(patch_arguments.start)                                        
                    case 7:
                        table_hit_high_cardioid(patch_arguments.start)
            case 3:
                match patch_arguments.note:
                    case 0:
                        sub_omni_noise(patch_arguments.start, 21)  
                    case 1:
                        sub_cardioid_noise(patch_arguments.start, 21) 
                    case 2:
                        low_omni_noise(patch_arguments.start, 21)
                    case 3:
                        low_cardioid_noise(patch_arguments.start, 21)    
                    case 4:
                        middle_omni_noise(patch_arguments.start, 21)    
                    case 5:
                        middle_cardioid_noise(patch_arguments.start, 21)   
                    case 6:
                        high_omni_noise(patch_arguments.start, 21) 
                    case 7:
                        high_cardioid_noise(patch_arguments.start, 21) 


my_handler = SampleNoteHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)              

class NoteLength(Enum):
    LONG = 1
    MIDDLE = 2
    SHORT = 3


sub_note_length_matrix = {
    NoteLength.LONG: {NoteLength.LONG: 0.6, NoteLength.MIDDLE: 0.4},
    NoteLength.MIDDLE: {NoteLength.LONG: 0.95, NoteLength.MIDDLE: 0.05}
}
sub_note_length_chain = MarkovChain(sub_note_length_matrix, NoteLength.LONG)

def sub_step_handler(i: int, time: float) ->  list[SequenceNote]:
    notes = []
    current_time = time
    end_time = current_time + 60
    while current_time < end_time:
        if random.choice([True, False]):
            notes.append(sub_omni_noise(current_time, 34))
        else:
            notes.append(sub_cardioid_noise(current_time, 34))
        current_time = current_time + (13 * random_range(0.85, 1.15))
    short_end_time = current_time + 21
    current_time = time
    
    while current_time < short_end_time:
        for _ in range(random_int_range(1, 2)):
            if random.choice([True, False]):
                notes.append(table_hit_sub_omni(current_time))
            else:
                notes.append(table_hit_sub_cardioid(current_time))
        match sub_note_length_chain.next():
            case NoteLength.LONG:
                current_time = current_time + (random_range(5, 8))
            case NoteLength.MIDDLE:
                current_time = current_time + (random_range(0.1, 0.3))
    return notes

low_note_length_matrix = {
    NoteLength.LONG: {NoteLength.LONG: 0.85, NoteLength.SHORT: 0.15},
    NoteLength.SHORT: {NoteLength.LONG: 0.95, NoteLength.SHORT: 0.05}
}
low_note_length_chain = MarkovChain(low_note_length_matrix, NoteLength.LONG)

def low_step_handler(i: int, time: float) ->  list[SequenceNote]:
    notes = []
    current_time = time
    end_time = current_time + 60
    while current_time < end_time:
        if random.choice([True, False]):
            notes.append(low_omni_noise(current_time, 34))
        else:
            notes.append(low_cardioid_noise(current_time, 34))
        current_time = current_time + (13 * random_range(0.85, 1.15))
    short_end_time = current_time + 21
    current_time = time
    
    while current_time < short_end_time:
        for _ in range(random_int_range(1, 2)):
            if random.choice([True, False]):
                notes.append(table_hit_low_omni(current_time))
            else:
                notes.append(table_hit_low_cardioid(current_time))
        match low_note_length_chain.next():
            case NoteLength.LONG:
                current_time = current_time + (random_range(3, 5))
            case NoteLength.SHORT:
                current_time = current_time + (random_range(0.05, 0.15))
    return notes




middle_note_length_matrix = {
    NoteLength.LONG: {NoteLength.LONG: 0.30, NoteLength.MIDDLE: 0.4, NoteLength.SHORT: 0.3},
    NoteLength.MIDDLE: {NoteLength.LONG: 0.50, NoteLength.MIDDLE: 0.2, NoteLength.SHORT: 0.3},
    NoteLength.SHORT: {NoteLength.LONG: 0.50, NoteLength.MIDDLE: 0.3, NoteLength.SHORT: 0.2},
}
middle_note_length_chain = MarkovChain(middle_note_length_matrix, NoteLength.MIDDLE)

def middle_step_handler(i: int, time: float) ->  list[SequenceNote]:
    notes = []
    current_time = time
    end_time = current_time + 60
    while current_time < end_time:
        if random.choice([True, False]):
            notes.append(middle_omni_noise(current_time, 34))
        else:
            notes.append(middle_cardioid_noise(current_time, 34))
        current_time = current_time + (13 * random_range(0.85, 1.15))
    short_end_time = current_time + 21
    current_time = time
    
    while current_time < short_end_time:
        for _ in range(random_int_range(1, 2)):
            if random.choice([True, False]):
                notes.append(table_hit_middle_omni(current_time))
            else:
                notes.append(table_hit_middle_cardioid(current_time))
        match middle_note_length_chain.next():
            case NoteLength.LONG:
                current_time = current_time + (random_range(2, 3))
            case NoteLength.MIDDLE:
                current_time = current_time + (random_range(0.1, 0.3))
            case NoteLength.SHORT:
                current_time = current_time + (random_range(0.01, 0.05))
    return notes



high_note_length_matrix = {
    NoteLength.LONG: {NoteLength.LONG: 0.20, NoteLength.MIDDLE: 0.4, NoteLength.SHORT: 0.4},
    NoteLength.MIDDLE: {NoteLength.LONG: 0.30, NoteLength.MIDDLE: 0.2, NoteLength.SHORT: 0.5},
    NoteLength.SHORT: {NoteLength.LONG: 0.30, NoteLength.MIDDLE: 0.3, NoteLength.SHORT: 0.4},
}
high_note_length_chain = MarkovChain(high_note_length_matrix, NoteLength.LONG)

def high_step_handler(i: int, time: float) ->  list[SequenceNote]:
    notes = []
    current_time = time
    end_time = current_time + 60
    while current_time < end_time:
        if random.choice([True, False]):
            notes.append(high_omni_noise(current_time, 34))
        else:
            notes.append(high_cardioid_noise(current_time, 34))
        current_time = current_time + (13 * random_range(0.85, 1.15))
    short_end_time = current_time + 21
    current_time = time
    
    while current_time < short_end_time:
        for _ in range(random_int_range(1, 2)):
            if random.choice([True, False]):
                notes.append(table_hit_high_omni(current_time))
            else:
                notes.append(table_hit_high_cardioid(current_time))
        match middle_note_length_chain.next():
            case NoteLength.LONG:
                current_time = current_time + (random_range(3, 5))
            case NoteLength.MIDDLE:
                current_time = current_time + (random_range(0.1, 0.5))
            case NoteLength.SHORT:
                current_time = current_time + (random_range(0.01, 0.05))
    return notes

PHI =  (1 + math.sqrt(5)) / 2
INV_PHI = 1 / PHI

class StartTime(Enum):
    EARLY = 1
    MIDDLE = 2
    LATE = 3

def early_time() -> float:
    return 60 - ((INV_PHI * 60) * random_range(0.85, 1.15))

def late_time() -> float:
    return (INV_PHI * 60) * random_range(0.85, 1.15)

start_time_matrix = {
    StartTime.EARLY: {StartTime.EARLY: 0.4, StartTime.LATE: 0.6},
    StartTime.LATE: {StartTime.EARLY: 0.6, StartTime.LATE: 0.4}
}
start_time_chain = MarkovChain(start_time_matrix, StartTime.LATE)

def make_sequence(handler: Callable[[int, float], list[SequenceNote]]):
    return Sequencer(1) \
        .add_step_handler(handler) \
        .next_time_handler(lambda i: 60)

class SequenceType(Enum):
    SUB = 1
    LOW = 2
    MIDDLE = 3
    HIGH = 4

sequence_type_matrix = {
    SequenceType.SUB: {SequenceType.LOW: 0.5, SequenceType.MIDDLE: 0.3, SequenceType.HIGH: 0.2},
    SequenceType.LOW: {SequenceType.SUB: 0.4, SequenceType.MIDDLE: 0.4, SequenceType.HIGH: 0.2},
    SequenceType.MIDDLE: {SequenceType.SUB: 0.2, SequenceType.LOW: 0.4, SequenceType.HIGH: 0.4},
    SequenceType.HIGH: {SequenceType.SUB: 0.2, SequenceType.LOW: 0.3, SequenceType.MIDDLE: 0.5}
}   

sequence_type_chain = MarkovChain(sequence_type_matrix, SequenceType.LOW)

high_sequence = Sequencer(1) \
    .add_step_handler(high_step_handler) \
    .next_time_handler(lambda i: 60)

middle_sequence = Sequencer(1) \
    .add_step_handler(middle_step_handler) \
    .next_time_handler(lambda i: 60)

low_sequence = Sequencer(1) \
    .add_step_handler(low_step_handler) \
    .next_time_handler(lambda i: 60) \

sub_sequence = Sequencer(1) \
    .add_step_handler(sub_step_handler) \
    .next_time_handler(lambda i: 60) \


last_sequence = None
for _ in range(NR_OF_PARTS):
    match sequence_type_chain.next():
        case SequenceType.SUB:
            handler = sub_step_handler
        case SequenceType.LOW:
            handler = low_step_handler
        case SequenceType.MIDDLE:
            handler = middle_step_handler
        case SequenceType.HIGH:
            handler = high_step_handler
    
    new_sequence = make_sequence(handler)
    
    match start_time_chain.next():
        case StartTime.EARLY:
            time_delta = early_time()
        case StartTime.LATE:
            time_delta = late_time()
    
    if last_sequence:
        new_sequence.spawn_sequencer(0, last_sequence)
        last_sequence.start_time_handler(lambda start: start + time_delta)
    last_sequence = new_sequence
sequence_notes = new_sequence.generate(0)
#sequence_notes = []
ui_piece = UiPieceBuilder() \
    .add_notes(sequence_notes) \
    .build()

piece_duration = ui_piece.get_duration()
min_freq, max_freq = ui_piece.get_track_min_max_freq()

piece_stats = { "total": piece_duration, "total minutes": piece_duration / 60.0}

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 200
NOTE_SCALE_FACTOR = 10
HEIGHT_INDENT = 180

ui_width = (200 + (piece_duration * NOTE_SCALE_FACTOR))
ui_height = (200 * len(ui_piece.tracks))

canvas = Canvas(width=ui_width, height=ui_height)
canvas.global_alpha = 0.7

for track_index, track in enumerate(ui_piece.tracks):
    canvas.font = "18px sans-serif"
    canvas.fill_style = "Black"
    canvas.fill_text(track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT)
    canvas.stroke_style = "Black"
    canvas.stroke_lines([(150, (track_index * TRACK_HEIGHT) + 10), (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10))])
    for note in track.notes:        
        relative_note =  ((note.freq - min_freq) / (max_freq - min_freq))    
        startx = 200 + (note.start * NOTE_SCALE_FACTOR)        
        starty = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT
        peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR        
        peaky = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT - 5
        endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
        endy = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT
        canvas.stroke_style = note.color
        canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])
canvas



Effect length 360.0


{'total': 232.29746810562693,
 'total minutes': 3.8716244684271155,
 'Sub noise': 209.52414867057337,
 'Sub sound': 208.23356586455034,
 'Middle noise': 232.29746810562693,
 'Middle sound': 231.72223162400107,
 'Low noise': 182.18087130320265,
 'Low sound': 186.08387987560033,
 'High noise': 142.63429145845342,
 'High sound': 144.62803666294073}

Canvas(height=1600, width=2522)

In [ ]:
piece.stop()
